Para começar um projeto novo

In [ ]:
scrapy startproject projectname


Para executar o shell

In [ ]:
scrapy shell 'url'

Este comando cria os seguintes diretórios.

In [ ]:
tutorial/
    scrapy.cfg            # deploy configuration file

    tutorial/             # project's Python module, you'll import your code from here
        __init__.py

        items.py          # project items definition file

        middlewares.py    # project middlewares file

        pipelines.py      # project pipelines file

        settings.py       # project settings file

        spiders/          # a directory where you'll later put your spiders
            __init__.py


Criando spiders! O arquivo deve ser gravado na pasta <i>/projectname/projectname/spiders/spidername.py</i>

In [ ]:
import scrapy


class QuotesSpider(scrapy.Spider):
    name = "quotes"

    def start_requests(self):
        urls = [
            'http://quotes.toscrape.com/page/1/',
            'http://quotes.toscrape.com/page/2/',
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        page = response.url.split("/")[-2]
        filename = 'quotes-%s.html' % page
        with open(filename, 'wb') as f:
            f.write(response.body)
        self.log('Saved file %s' % filename)

A variável name deve levar um nome único. De modo que possam ser criadas várias spiders no mesmo projeto, e que elas possam ser chamadas do terminal de comando executando :

In [ ]:
scrapy crawl spidername

Na função start_request(self) será colocado as urls de onde as spiders irão extrair as informações.

Na função parse será definido de que parte daquela página será extraído os dados. Para isso é preciso abrir a página e estudar seu código fonte (HTML). Caso queiramos extrair todas as imagens da página, precisamos olhar o em que parte do código estão localizadas as imagens. No exemplo que aparecerá mais a frente, serão extraídos trechos de frases escritas por diversos autores. Esses textos estão entre tags <b>div</b>, <b>class quote</b>. 

No exemplo acima, QuotesSpiders, o código diz para salvar as informações extraídas em dois arquivos html,cada um com um nome diferente que será gerado de acordo com o número da página.

Cada "quote" estará em uma estrutura html como a seguinte:

<div class="quote">
    <span class="text">“The world as we have created it is a process of our
    thinking. It cannot be changed without changing our thinking.”</span>
    <span>
        by <small class="author">Albert Einstein</small>
        <a href="/author/Albert-Einstein">(about)</a>
    </span>
    <div class="tags">
        Tags:
        <a class="tag" href="/tag/change/page/1/">change</a>
        <a class="tag" href="/tag/deep-thoughts/page/1/">deep-thoughts</a>
        <a class="tag" href="/tag/thinking/page/1/">thinking</a>
        <a class="tag" href="/tag/world/page/1/">world</a>
    </div>
</div>

In [ ]:
response.css('div.quote')

In [ ]:
quote = response.css('div.quote')

In [ ]:
text = quote.css("span.text::text").get()

In [ ]:
author = quote.css("small.author::text").get()

In [ ]:
tags = quote.css("div.tags a.tag::text").getall()

In [ ]:
for quote in response.css("div.quote"):
    text = quote.css("span.text::text").get()
    author = quote.css("small.author::text").get()
    tags = quote.css("div.tags a.tag::text").getall()
    print(dict(text=text, author=author, tags=tags))
    

Mais um exemplo de spider. Importante lembrar que cada spider deve ter seu nome único.

In [ ]:
import scrapy


class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        'http://quotes.toscrape.com/page/1/',
        'http://quotes.toscrape.com/page/2/',
    ]

    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').get(),
                'author': quote.css('small.author::text').get(),
                'tags': quote.css('div.tags a.tag::text').getall(),
            }

 Note que a função parse() devolve um dicionário!


Para registrar os dados dentro de um arquivo é necessário executar o scrapy com o seguinte comando:

In [ ]:
scrapy crawl quotes -o quotes.json

Vou agora testar o exemplo acima no site do portal da cidade de guaxupé. Quero extrair as manchetes da página de notícia.E quero testar de 15 em 15 dias para estudar o funcionamento do spider, quando houver mudanças no site, como atualização de notícias! Ele pegará as notícias novas ? ou precisaria reestruturar o código. Quando os sites realizam atualizações, como a inserção de novas notícias, eles mantém a estrutura em html? ou o código muda?

In [ ]:
.js-results  #CSS selector Esse é onde estão todas as manchetes

In [ ]:
<body>
    <main> 
        <section class='container'>
            <section class = 'main-content--flex'>
                <article class='list-content'>
                    <div class='js-results'>
                        <a class='list-content--item' href='linkdecadamanchete'> </a>
                            <div class='list-content--description'>
                                #O título da notícia 
                                <h2 class='list-content--title grey-text text-darken-4'> </h2>
                                #O subtítulo da notícia
                                <p class='news-item--subtitle text-sm grey-text text-darken-2'> </p>
                                #A data da notícia
                                <p class='news-item--post-date text-xs grey-text text-darken-1'> </p>
                                
                        <a class='list-content--item' href='linkdecadamanchete'> </a>
                        <a class='list-content--item' href='linkdecadamanchete'> </a>
                        <a class='list-content--item' href='linkdecadamanchete'> </a>
                    </div>
                </article>
            </section>
        </section>
    </main>
</body>



                    

Esse é o código html onde estão as primeiras manchetes

In [ ]:
import scrapy

In [ ]:
noticias = response.css('a.list-content--item')

O código acima irá gerar uma lista imensa de selectors,cada um correspondendo a uma manchete diferente. A idéia aqui seria extrair o título, o subtítulo,a data e o link da notícia, e criar inicialmente um dicionário.
Para fazer isso, basta eu iterar por elas e extrair cada dado.Depois posso usar o dicionário para criar uma tabela com o panda.

In [ ]:
for n in range(len(noticias)):
    yield {
            'título': noticias[n].css('h2.list-content--title::text').get(),
            'subtítulo': noticias[n].css('p.news-item--subtitle text-sm grey-text text-darken-2::text').get(),
            'data': noticias[n].css('p.news-item--post-date text-xs grey-text text-darken-1::text').get(),
            }

In [ ]:
titulo = noticias[0].css('h2.list-content--title::text').get()

O trecho html do título estará escrito assim,dentro da class <b>list-content--title grey-text text-darken-4</b>. Mas só preciso pegar 'h2.list.content--title'

In [ ]:
subtitulo = noticias[0].css('p.news-item--subtitle::text').get()

In [ ]:
data = noticias[0].css('p.news-item--post-date text-xs grey-text text-darken-1::text').get()

Para criar uma spider, existe a possibilidade de executar o comando a seguir, que criará um modelo simples de spider, a ser preenchido.

In [ ]:
scrapy genspider basic web

Fiz o teste com os códigos acima. Criei um projeto chamado guaxupe, os itens, o spider e coloquei as informações extraídas em um arquivo .json. deu certo!

Mas preciso limpar as informações. colocar em algum lugar do código, a função .encode('utf-8') pra tratar as strings. Também tenho a intenção de extrair os links e criar uma tabela com o panda, só pra treinar.

E depois preciso testar o spider a cada vez que inserirem mais notícias.

Seguindo com o exemplo do spider Quote, proposto pelo tutorial do site oficial do scrapy.

Caso eu queira pegar informações de todo o site, ao invés de extrair as informações de apenas uma página. Eu precisaria então acessar os links. Precisamos então olhar o código html da página e descobrir onde se encontram os links. Segue abaixo um link da página do exemplo:

In [ ]:
<ul class="pager">
    <li class="next">
        <a href="/page/2/">Next <span aria-hidden="true">&rarr;</span></a>
    </li>
</ul>

Para extrair esse link devemos colocar esse <i><b>::attr(href)</b></i> ao caminho.

In [ ]:
In[1] response.css('li.next a::attr(href)').get()
In[1] '/page/2/'

Ou então utilizar a propriedade atributo disponível. <b><i>.attrib('href')</i></b>

In [ ]:
In[1]    response.css('li.next a').attrib['href']
Out [1]  '/page/2/'

Abaixo,segue modificações no spider, para que ele possa acessar links.

In [ ]:
import scrapy


class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        'http://quotes.toscrape.com/page/1/',
    ]

    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').get(),
                'author': quote.css('small.author::text').get(),
                'tags': quote.css('div.tags a.tag::text').getall(),
            }

        next_page = response.css('li.next a::attr(href)').get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

Depois de extrair as informações da página onde ele se encontra, o spider vai atrás do link para a próxima página e utilizando o método urljoin() e solicita um <b>request</b>, se regitrando como callback, ligado ao método parse para extrair as informações da próxima página.E continuar extraindo informações até que next_page seja None.



Para acessar links, é preciso criar regras de acordo com o site em que estamos, e também referente às informações que queremos extrair. No caso acima, criamos um script que acessa a página seguinte, e vai procurando por novos links, até que não haja mais 'próxima página'. Ele seria ideal para extrair informações de blogs, foruns e outros sites com paginação.

In [ ]:
next_page = response.css('li.next a::attr(href)').get()
        if next_page is not None:
            yield response.follow(next_page, callback=self.parse)

Diferente de <i>scrapy.Request</i>, <i> response.follow</i> suporta urls relativos diretamente. Não precisa chamar urljoin . Note qie response.follow só retorna uma instância Request, você ainda precisa yield a Request.

In [ ]:
for href in response.css('ul.pager a::attr(href)'):
    yield response.follow(href, callback=self.parse)

Posso utilizar o módulo json para trabalhar com as informações contidas em um arquivo .json. Anteriormente, havíamos extraído as manchetes de um site de notícia e havíamos armazenado em um dicionário e colocado tudo dentro de um arquivo .json. O próximo passo seria trabalhar com esses dados. Utilizá-los. Para isso podemos fazer o seguinte:

In [ ]:
import jason
json_file = open("/home/file.json","r",encoding = 'utf-8')
file = json.load(json_file)
json_file.close()

Se fizermos uma chamada file, no console do python, depois de ter executado os códigos acima, aparecerá na tela um dicionário,contendo todas as informações que extraímos.

    Podemos depois utilizar o módulo pandas, para fazer um data frame com as informações

In [ ]:
import pandas
df = pd.DataFrame(data=file


Se executarmos df, no console python, após os códigos acima, teremos um dataframe com as informações coletadas.Mas como foram muitos dados coletados, e os subtítulos da manchetes tem um tamanho considerável, não vai caber tudo na tela. Um próximo ponto para eu investigar seria como transformar esse dataframe em uma planilha do excel, por exemplo.

Fico imaginando como poderia utilizar essa ferramenta, pensando em uma empresa. Uma possibilidade, que foi apresentada no vídeo do coursera, é fazer uma pesquisa de preços com os concorrentes da empresa, para poder estabelecer o menor preço do mercado. 

Ou então poderia fazer um aplicativo que receberia informações, como por exemplo as manchetes de um jornal. O aplicativo seria atualizado automaticamente, a medida que o spider fosse executado e extraísse novo conteúdo.

Projeto spider 2 - dicionário com CNAEs


In [ ]:
https://blog.meifacil.com/quero-ser-mei/atividades-mei-2020-quais-atividades-podem-ter-cnpj-mei/

In [ ]:
a = response.css('table.table')
b = a.css('tbody')
c = b.css('tr')